In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.transforms import v2

[MRI Scans Alzeimer Detection Dataset (via Hugging Face)](https://huggingface.co/datasets/yogitamakkar178/mri_scans_alzeimer_detection)

In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("yogitamakkar178/mri_scans_alzeimer_detection")

Resolving data files:   0%|          | 0/9552 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1279 [00:00<?, ?it/s]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True)
])
def preprocess(data):
    data["image"] = [transform(image) for image in data["image"]]
    return data
ds = ds.with_format("torch") # ds = ds.with_format("torch", device=device)
ds = ds.with_transform(preprocess)

In [4]:
train_loader = DataLoader(ds["train"], shuffle=True, batch_size=32)
test_loader = DataLoader(ds["test"], shuffle=True, batch_size=32)

In [5]:
class AlzeimersModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) # 1x(128x128) -> 32x(128x128)
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)# 32x(128x128) -> 32x(64x64)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 32x(64x64) -> 64x(64x64)
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # 64x(64x64) -> 64x(32x32)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 32x(64x64) -> 64x(64x64)
        self.relu3 = nn.LeakyReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # 64x(64x64) -> 64x(32x32)

        self.fc1   = nn.Linear(128 * 16 * 16, 128 * 16) # flatten
        self.dropout1 = nn.Dropout(0.4)
        self.fc2   = nn.Linear(128 * 16, 4)

    def forward(self, x):
        # first pass
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        # second pass
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        # third pass
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        # flatten and classify
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        
        return x

In [6]:
model = AlzeimersModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [7]:
def train_model(model, optimizer, criterion, device, train_loader, epoch):
    model.train()
    for idx, batch in enumerate(train_loader):
        image = batch["image"].to(device)
        target = batch["label"].to(device)
        outputs = model(image)
        loss = criterion(outputs, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if idx % 25 == 0:
            print(f"Epoch [{epoch}].[{idx}] Loss: {loss}")

    with torch.no_grad():
        num_correct = 0
        num_total = 0
        for batch in train_loader:
            image = batch["image"].to(device)
            target = batch["label"].to(device)
            outputs = model(image)
            output = outputs.argmax(dim=1)
            num_correct += (output == target).sum().item()
            num_total += target.size(0)
    accuracy = num_correct / num_total
    print("")
    print(f"Epoch [{epoch}] Train accuracy: {accuracy}")

def test_model(model, device, test_loader, epoch):
    with torch.no_grad():
        num_correct = 0
        num_total = 0
        for batch in test_loader:
            image = batch["image"].to(device)
            target = batch["label"].to(device)
            outputs = model(image)
            output = outputs.argmax(dim=1)
            num_correct += (output == target).sum().item()
            num_total += target.size(0)
        accuracy = num_correct / num_total
        print("")
        print(f"Epoch [{epoch}] Test accuracy: {accuracy}")
        print("")

In [8]:
num_epochs = 20 # hyperparameter
for epoch in range(1, num_epochs + 1):
    train_model(model, optimizer, criterion, device, train_loader, epoch)
    test_model(model, device, test_loader, epoch)

Epoch [1].[0] Loss: 1.3855810165405273
Epoch [1].[25] Loss: 1.3221006393432617
Epoch [1].[50] Loss: 1.2587213516235352
Epoch [1].[75] Loss: 0.9080191850662231
Epoch [1].[100] Loss: 0.8253974914550781
Epoch [1].[125] Loss: 0.7887484431266785
Epoch [1].[150] Loss: 0.7643625140190125
Epoch [1].[175] Loss: 0.9295473098754883
Epoch [1].[200] Loss: 0.6828395128250122
Epoch [1].[225] Loss: 0.41368886828422546
Epoch [1].[250] Loss: 0.5650813579559326
Epoch [1].[275] Loss: 0.3309713304042816

Epoch [1] Train accuracy: 0.7949120603015075

Epoch [1] Test accuracy: 0.5222830336200156

Epoch [2].[0] Loss: 0.24613016843795776
Epoch [2].[25] Loss: 0.36898088455200195
Epoch [2].[50] Loss: 0.272490918636322
Epoch [2].[75] Loss: 0.5769102573394775
Epoch [2].[100] Loss: 0.4156542420387268
Epoch [2].[125] Loss: 0.29828163981437683
Epoch [2].[150] Loss: 0.48841753602027893
Epoch [2].[175] Loss: 0.32642698287963867
Epoch [2].[200] Loss: 0.4284372329711914
Epoch [2].[225] Loss: 0.2629874348640442
Epoch [2].[

# Reflection
### Results
The best test accuracy obtained was around 92%. The test accuracy tended to be consistently about 10% below the training accuracy, which reached close to 100% accuracy. This suggests the training set was overfitted at some point.
### Improvements
The outcome of the Alzeimers CNN could have potentially been better if slight Data Augmentation was implemented (slight rotation, change in contrast and brightness, added blurness/sharpness, etc.), in order to add more noise to generalize the dataset more.

A better dataset could also have yielded better results. The [dataset](https://huggingface.co/datasets/yogitamakkar178/mri_scans_alzeimer_detection) used is relatively small (~10.8k entries), has differently balanced train and test sets, and has a very imbalanced test set. A more balanced and larger dataset could have definitely improved the test accuracy.

Implementing normalization in the data transformation pipeline could have helped the neural network, specifically the optimization algorithm.